<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def f(Re, eD):
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD))**16
    t2 = (37530/Re)**16
    return 2* ((8/Re)**12 + 1/(t1+t2)**1.5)**(1/12.)

In [3]:
f(1600,1e-3)

0.010000004435664686

In [4]:
16/1600

0.01

In [5]:
f(1e5,0.002)

0.0063322857085468835